In [1]:
# standard imports
import numpy as np
import matplotlib.pyplot as plt 
import datetime

#database imports
from snowexsql.db import get_db
from snowexsql.data import PointData, LayerData, ImageData, SiteData
from snowexsql.conversions import query_to_geopandas

In [2]:
# load the database
db_name = 'snow:hackweek@52.32.183.144/snowex'
engine, session = get_db(db_name)

print('snowexsql database loading successfull!')

snowexsql database loading successfull!


In [3]:
# Query the session using .surveyors() to generate a list
qry = session.query(ImageData.surveyors)

# Locate all that are distinct
airborne_sensors_list = session.query(ImageData.surveyors).distinct().all()

print('list of airborne sensors by "surveyor" name: \n', airborne_sensors_list)

list of airborne sensors by "surveyor" name: 
 [('USGS',), ('UAVSAR team, JPL',), ('ASO Inc.',)]


In [4]:
# Airborne sensor from list above
sensor = 'UAVSAR team, JPL'

# Form on the Images table that returns Raster collection dates
qry = session.query(ImageData.date)

# Filter for UAVSAR data
qry = qry.filter(ImageData.surveyors == sensor)

# Grab the unique dates
qry = qry.distinct()

# Execute the query 
dates = qry.all() 

# Clean up the dates 
dates = [d[0] for d in dates] 
dlist = [str(d) for d in dates]
dlist = ", ".join(dlist)
print('%s flight dates are: %s' %(sensor, dlist))

# Find all the snow pits done on these days
qry = session.query(SiteData.geom, SiteData.site_id, SiteData.date)
qry = qry.filter(SiteData.date.in_(dates))

# Return a geopandas df
df = query_to_geopandas(qry, engine)

# View the returned pandas dataframe!
print(df.head())

# Close your session to avoid hanging transactions
session.close()

UAVSAR team, JPL flight dates are: 2020-01-31, 2020-02-12
                             geom site_id        date
0  POINT (740652.000 4327445.000)     2C2  2020-01-31
1  POINT (744396.000 4323540.000)    8C26  2020-01-31
2  POINT (741960.000 4326644.000)    6C10  2020-01-31
3  POINT (741493.000 4326833.000)     1C8  2020-01-31
4  POINT (745340.000 4322754.000)    8S28  2020-01-31


In [5]:
# Pick a day from the list of dates
dt = dates[0] 

# Find all the snow pits done on these days 
qry = session.query(SiteData.geom, SiteData.site_id, SiteData.date)
qry = qry.filter(SiteData.date == dt)

# Return a geopandas df
df_exact = query_to_geopandas(qry, engine)

print('%s pits overlap with %s on %s' %(len(df_exact), sensor, dt))

# View snows pits that align with first UAVSAR date
df_exact.head()

17 pits overlap with UAVSAR team, JPL on 2020-01-31


,geom,site_id,date
0,POINT (740652.000 4327445.000),2C2,2020-01-31
1,POINT (744396.000 4323540.000),8C26,2020-01-31
2,POINT (741960.000 4326644.000),6C10,2020-01-31
3,POINT (741493.000 4326833.000),1C8,2020-01-31
4,POINT (745340.000 4322754.000),8S28,2020-01-31


In [7]:
df_exact

,geom,site_id,date
0,POINT (740652.000 4327445.000),2C2,2020-01-31
1,POINT (744396.000 4323540.000),8C26,2020-01-31
2,POINT (741960.000 4326644.000),6C10,2020-01-31
3,POINT (741493.000 4326833.000),1C8,2020-01-31
4,POINT (745340.000 4322754.000),8S28,2020-01-31
5,POINT (744862.000 4323250.000),4C30,2020-01-31
6,POINT (742466.000 4324372.000),2N12,2020-01-31
7,POINT (744477.000 4323731.000),8C22,2020-01-31
8,POINT (740765.000 4327379.000),2C3,2020-01-31
9,POINT (741378.000 4326992.000),1C7,2020-01-31


In [8]:
# Find all the data that was collected on 1-31-2020
dt = datetime.date(2020, 1, 31)

#--------------- Point Data -----------------------------------
# Grab all Point data instruments from our date
point_instruments = session.query(PointData.instrument).filter(PointData.date == dt).distinct().all()
point_type = session.query(PointData.type).filter(PointData.date == dt).distinct().all()

# Clean up point data (i.e. remove tuple)
point_instruments = [p[0] for p in point_instruments]
point_instruments = ", ".join(point_instruments)
point_type = [p[0] for p in point_type]
point_type = ", ".join(point_type)
print('Point data on %s are: %s, with the following list of parameters: %s' %(str(dt), point_instruments, point_type))

#--------------- Layer Data -----------------------------------
# Grab all Layer data instruments from our date
layer_instruments = session.query(LayerData.instrument).filter(LayerData.date == dt).distinct().all()
layer_type = session.query(LayerData.type).filter(LayerData.date == dt).distinct().all()

# Clean up layer data 
layer_instruments = [l[0] for l in layer_instruments if l[0] is not None]
layer_instruments = ", ".join(layer_instruments)
layer_type = [l[0] for l in layer_type]
layer_type = ", ".join(layer_type)
print('\nLayer Data on %s are: %s, with the following list of parameters: %s' %(str(dt), layer_instruments, layer_type))

#--------------- Image Data -----------------------------------
# Grab all Image data instruments from our date
image_instruments = session.query(ImageData.instrument).filter(ImageData.date == dt).distinct().all()
image_type = session.query(ImageData.type).filter(ImageData.date == dt).distinct().all()

# Clean up image data (i.e. remove tuple)
image_instruments = [i[0] for i in image_instruments]
image_instruments = ", ".join(image_instruments)
image_type = [i[0] for i in image_type]
image_type = ", ".join(image_type)
print('\nImage Data on %s are: %s, with the following list of parameters: %s' %(str(dt), image_instruments, image_type))

Point data on 2020-01-31 are: camera, magnaprobe, pit ruler, with the following list of parameters: depth

Layer Data on 2020-01-31 are: IRIS, snowmicropen, IS3-SP-11-01F, with the following list of parameters: sample_signal, force, grain_size, density, reflectance, permittivity, lwc_vol, manual_wetness, equivalent_diameter, specific_surface_area, grain_type, temperature, hand_hardness

Image Data on 2020-01-31 are: UAVSAR, L-band InSAR, with the following list of parameters: insar amplitude


In [9]:
def parse_veg_class(site_id):
    
    '''
    This function parses snow pit data into three vegetation classes:
        - 1). Treeless, 2). Sparce, and 3). Dense
        
    It uses a python dictionary where:
        (k) keys: are the vegetation classes
        (v) values: are the first digit in the pitID assignment

    
    '''
    
    # Classifying by vegetation coverage 
    veg_class = {'treeless':[1, 2, 3], 'sparse':[4, 5, 6], 'dense':[7, 8, 9]}
     
    vclass = None 
    
    class_id = site_id[0]
    
    if class_id.isnumeric():
        class_id = int(class_id)

        for k,v in veg_class.items():

            if class_id in v: #if the first digit in the site_id is 'v' assign it to the corresponding 'k'
                vclass = k 
                
    return vclass 

In [10]:
def parse_depth_class(site_id):
    
    '''
    This function parses snow pit data into three depth classes:
        - 1). Shallow, 2). Medium, and 3). Deep
        
    It uses a python dictionary where:
        (k) keys: are the depth classes
        (v) values: are the first digit in the pitID assignment
      
  
    '''
        
    # Classifying by 2017 depth 
    depth_class = {'shallow':[1, 4, 7], 'medium':[2, 5, 8], 'deep':[3, 6, 9]} 
   
    dclass = None 
    
    class_id = site_id[0]
    
    if class_id.isnumeric(): #for the outlier TS site
        class_id = int(class_id) #cast as integer

        for k,v in depth_class.items(): #for the key, value pairs in the dict listed above:

            if class_id in v: #if the first digit in the site_id is 'v' assign it to the corresponding 'k'
                dclass = k 

    return dclass 

In [11]:
# Load the database
db_name = 'snow:hackweek@52.32.183.144/snowex'
engine, session = get_db(db_name)

# Query for Layer Data
result = session.query(LayerData.type).distinct().all()

# Filter for density data
qry = session.query(LayerData).filter(LayerData.type=='density')

# Form our dataframe from the query 
df = query_to_geopandas(qry, engine)
df['value'] = df['value'].astype(float)  #cast the value as a float (they are strings)

# Parse snow pit data by the veg/depth matrix
df['veg_class'] = [parse_veg_class(i) for i in df['site_id']] #run the parse_veg function for every site_id
df['depth_class'] = [parse_depth_class(i) for i in df['site_id']] #run the parse_depth funciton for every site_id

# Select columns of interest
col_list = ['site_id', 'date', 'type', 'latitude',
       'longitude', 'depth', 'value', 'veg_class', 'depth_class']
df = df[col_list]

# View a sample --> notice parsed veg_class and depth_class columns were added!
df.sample(5)

,site_id,date,type,latitude,longitude,depth,value,veg_class,depth_class
1412,8N35,2020-02-10,density,39.032460,-108.170096,107.0,201.0,dense,medium
1170,1N3,2020-02-11,density,39.033787,-108.218511,67.0,242.5,treeless,shallow
1755,9C16,2020-02-05,density,39.044273,-108.197050,79.0,271.0,dense,deep
2011,6N46,2020-02-01,density,39.028131,-108.151811,47.0,284.5,sparse,deep
930,3S14,2020-02-01,density,39.019233,-108.185671,103.0,219.0,treeless,deep


In [15]:
df.site_id

0       9C17
1       9C17
2       9C17
3       9C17
4       9C17
        ... 
2817    3N26
2818    3N26
2819    3N26
2820    1C14
2821    1C14
Name: site_id, Length: 2822, dtype: object

In [16]:
df.date

0       2020-01-30
1       2020-01-30
2       2020-01-30
3       2020-01-30
4       2020-01-30
           ...    
2817    2020-02-08
2818    2020-02-08
2819    2020-02-08
2820    2020-01-31
2821    2020-01-31
Name: date, Length: 2822, dtype: object

In [17]:
# Group by site-id to count classes
gb = df.groupby(['site_id', 'veg_class', 'depth_class'])

print(gb['site_id'].count().groupby('veg_class').count())
print('\n')
print(gb['site_id'].count().groupby('depth_class').count())

veg_class
dense       50
sparse      39
treeless    60
Name: site_id, dtype: int64


depth_class
deep       45
medium     78
shallow    26
Name: site_id, dtype: int64


In [18]:
# import libraries
import re
import zipfile
import getpass
from osgeo import gdal 
import os  # for chdir, getcwd, path.basename, path.exists
import pandas as pd # for DatetimeIndex
import codecs # for text parsing code
import netrc
import rasterio as rio
import glob

In [19]:
# Get NASA EARTHDATA Credentials from ~/.netrc or manual input
try:
    os.chmod('/home/jovyan/.netrc', 0o600) #only necessary on jupyterhub
    (ASF_USER, account, ASF_PASS) = netrc.netrc().authenticators("urs.earthdata.nasa.gov")
except:
    ASF_USER = input("Enter Username: ")
    ASF_PASS = getpass.getpass("Enter Password: ")

In [20]:
# directory in which the notebook resides
if 'tutorial_home_dir' not in globals():
     tutorial_home_dir = os.getcwd()
print("Notebook directory: ", tutorial_home_dir)

if not os.path.exists('/tmp/'):
    os.chdir('/tmp')
   
# directory for data downloads

data_dir = os.path.join('/tmp')
os.makedirs(data_dir, exist_ok=True)
print(data_dir)

Notebook directory:  /home/jovyan/uavsar/ShazWork
/tmp


In [21]:
%%time 

files = ['https://datapool.asf.alaska.edu/INTERFEROMETRY/UA/grmesa_27416_20003-028_20008-004_0018d_s01_L090_01_int.zip,]
    
for file in files:
    print(f'downloading {file}...')
    filename = os.path.basename(file)
    
    if not os.path.exists(os.path.join(data_dir,filename)):
        cmd = "wget -q {0} --user={1} --password={2} -P {3} -nc".format(file, ASF_USER, ASF_PASS, data_dir)
        os.system(cmd)
    else:
        print(filename + " already exists. Skipping download ..")
print("done")

SyntaxError: EOL while scanning string literal (<unknown>, line 1)

In [23]:
!aws s3 ls s3://snowex-data/uavsar-project/UAVSAR_images/

                           PRE GrMesa_2015_054_261_0006d/
                           PRE GrMesa_2015_055_081_0008d/
                           PRE GrMesa_2015_055_261_0008d/
                           PRE GrMesa_2015_060_081_0048d/
                           PRE GrMesa_2015_060_261_0048d/
                           PRE GrMesa_2015_100_081_0360d/
                           PRE GrMesa_2016_095_081_0313d/
                           PRE GrMesa_2016_095_261_0313d/
                           PRE grmesa_2017_002_078_0016d/
                           PRE grmesa_2017_002_078_0019d/
                           PRE grmesa_2017_002_078_0030d/
                           PRE grmesa_2017_002_078_0053d/
                           PRE grmesa_2017_002_080_0016d/
                           PRE grmesa_2017_002_080_0030d/
                           PRE grmesa_2017_002_080_0053d/
                           PRE grmesa_2017_002_260_0016d/
                           PRE grmesa_2017_002_260_0019d/
              

In [25]:
!aws s3 cp s3://snowex-data/uavsar-project/UAVSAR_images/grmesa_2020_003_274_0011d/grmesa_27416_20003-028_20005-007_0011d_s01_L090HH_01.cor.grd.tiff /tmp/grmesa_27416_20003-028_20005-007_0011d_s01_L090HH_01.cor.grd.tiff

fatal error: An error occurred (404) when calling the HeadObject operation: Key "uavsar-project/UAVSAR_images/grmesa_2020_003_274_0011d/grmesa_27416_20003-028_20005-007_0011d_s01_L090HH_01.cor.grd.tiff" does not exist
